In [48]:
#Basically I have to extract the right bedroom word....
from bs4 import BeautifulSoup

In [93]:
# filePath = r"D:\propertyimages\webImages\531 W. Street Road - Gable _ County Builders Inc.html"
# filePath = r"D:\propertyimages\webImages\1130 - Quality Design Homes.html"
# filePath = r"D:\propertyimages\webImages\1272 Bentley Rd. [Contract Pending] - Gateway Construction.html"
filePath = r"D:\propertyimages\webImages\1382 Ashwood Lane, Victor, NY 14564 (MLS # R1389827) _ Morrell Builders.html"
# filePath = r"D:\propertyimages\webImages\1704 N. Harrison Street.html"
# filePath = r"D:\propertyimages\webImages\3308 Club Court _ New Homes in Naperville IL _ Eastman Properties.html"
# filePath = r"D:\propertyimages\webImages\7138 Cassidy Court #Lot 219, Victor, NY 14564 (MLS # R1389188) _ Morrell Builders.html"
# filePath = r"D:\propertyimages\webImages\Atlantic - Schmidt Homes, LLC.html"
# filePath = r"D:\propertyimages\webImages\CAREFREE LIVING IN THE ENCLAVE AT AUGUSTA GREEN - Spencer Homes, LLC.html"
# filePath = r"D:\propertyimages\webImages\Chelsea Park.html"
# filePath = r"D:\propertyimages\webImages\Francis ADU 896 _ Skagit Design Homes.html"
# filePath = r"D:\propertyimages\webImages\Home _ Pate Jones Construction.html"
# filePath = r"D:\propertyimages\webImages\Home Plans - Severyn Development.html"
# filePath = r"D:\propertyimages\webImages\House Plan 1430 - Gough Homes.html"
# filePath = r"D:\propertyimages\webImages\Huntington - Newport Homes.html"
# filePath = r"D:\propertyimages\webImages\Lake Winnebago _ Pfeifer Homes Inc. _ Lee's Summit Kansas City.html"
# filePath = r"D:\propertyimages\webImages\McCabe Floor Plan _ Heartland Builders.html"
# filePath = r"D:\propertyimages\webImages\Riviera _ Pfeifer Homes Inc. _ Lee's Summit kansas City.html"
# filePath = r"D:\propertyimages\webImages\Saddle Ridge - Gateway Construction.html"
# filePath = r"D:\propertyimages\webImages\THE DAWSON - Colorado Springs Custom Homes by Jayden Homes.html"
# filePath = r"D:\propertyimages\webImages\The Westbrook _ Inspired Homes.html"
# filePath = r""

In [94]:
#parsed response is stored in the soup object...
soup = ''
with open(filePath, encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

In [95]:
all_tags = soup.body.find_all()

req_tags = []

ignored_tags = ['script', 'style', 'wix-image']

main_tags = []

#req_tags will contain the tags except the above mentioned tags...
for tag in all_tags:
    if(tag.name not in ignored_tags):
        req_tags.append(tag)

#main_tags will only contain the tags which contain the tags only having 1 descendant....
for tag in req_tags:
    if(len(list(tag.descendants)) == 1):
        main_tags.append(tag)

keyword_tags = []

#The below code append keywords if any in the webpage..
for tag in main_tags:
    content = tag.text.strip()
    if(content):
        content = content.lower()
        if('bed' in content):
            keyword_tags.append(tag)

if(len(keyword_tags) == 0):
    for tag in main_tags:
        content = tag.text.strip()
        if(content):
            content = content.lower()
            if('bath' in content):
                keyword_tags.append(tag)

                


In [96]:
src_list = []

numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

#Finding the proper anchor point or proper keyword....
for tag in keyword_tags:
    content = tag.text.strip()
    flag = 0
    #Checking if the number is present in the string....
    for number in numbers:
        if(number in content):
            flag = 1
            break
    #Checking if the number string is presrent as next sibling..
    if(flag == 0 and tag.next_sibling):
        if(isinstance(tag.next_sibling, str)):
            for number in numbers:
                if(number in tag.next_sibling):
                    flag = 1
                    break
    #Checking if the number string is present as previous sibling...
    if(flag == 0 and tag.previous_sibling):
        if(isinstance(tag.next_sibling, str)):
            for number in numbers:
                if(number in tag.next_sibling):
                    flag = 1
                    break
    #Checking if the number is present in the next sibling tag...
    if(flag == 0 and tag.findNext()):
        if(isinstance(tag.findNext(), str)):
            for number in numbers:
                if(number in tag.findNext()):
                    flag = 1
                    break
        else:
            tempContent = tag.findNext().text.strip()
            if(tempContent):
                for number in numbers:
                    if(number in tempContent):
                        flag = 1
                        break
    #Checking if the number is present in the previous tag...
    if(flag == 0 and tag.findPrevious()):
        if(isinstance(tag.findPrevious(), str)):
            for number in numbers:
                if(number in tag.findPrevious()):
                    flag = 1
                    break
        else:
            tempContent = tag.findPrevious().text.strip()
            if(tempContent):
                for number in numbers:
                    if(number in tempContent):
                        flag = 1
                        break


    ignored_words = ['icon', 'facebook', 'twitter', 'instagram', 'logo', 'map', 'pdf']

    
    if(flag):
        # print(flag)
        img_list = []
        parent = ''
        store = tag
        #This loop will keep on executing when the length of images list is empty and there is some tag...
        while(len(img_list) == 0 and store):
            x = store.previous_sibling
            #While there is a sibling, then we shall keep searching the image...
            while(x):
                if(isinstance(x, str) == False):
                    images = x.find_all('img')
                    if(len(images) != 0):
                        for img in images:
                            marker = 0
                            for itr in ignored_words:
                                if (itr in img.get('src').lower()):
                                    marker = 1
                                    break
                            if(marker == 0):
                                img_list.append(img)
                                    
                x = x.previous_sibling

            #We shall find the images in the parent tag..
            images = store.find_all('img')
            if(len(images) != 0):
                for img in images:
                    marker = 0
                    for itr in ignored_words:
                        if(itr in img.get('src').lower()):
                            marker = 1
                            break
                    if(marker == 0):
                        img_list.append(img)

            store = store.parent

        #Appending the src into the source list....
        if(len(img_list) != 0):
            for x in img_list:
                src_list.append(x.get('src'))
        # break


In [97]:
src_list = list(set(src_list))
len(src_list)

24

In [98]:
for source in src_list:
    print(source)

https://images.marketleader.com/HouseImages/CENTRALNY/827/R1389827_12.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketleader.com/HouseImages/CENTRALNY/827/f_R1389827_09.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketleader.com/HouseImages/CENTRALNY/827/R1389827_08.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketleader.com/HouseImages/CENTRALNY/827/R1389827_06.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketleader.com/HouseImages/CENTRALNY/827/R1389827_03.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketleader.com/HouseImages/CENTRALNY/827/R1389827_09.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketleader.com/HouseImages/CENTRALNY/827/R1389827_07.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketleader.com/HouseImages/CENTRALNY/827/R1389827_16.jpg?PhotoAccessKey=dd7012f37dc79c5cf7da00cb6877dc45
https://images.marketl